<a href="https://colab.research.google.com/github/andresenrique08/ML_2020/blob/master/Proyecto_Malware/ProyectoFinalML_trainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Texto de título predeterminado
ruta_train = "/content/drive/MyDrive/LleyZon Emoxito/train.csv" #@param {type:"string"}
ruta_test = "/content/drive/MyDrive/LleyZon Emoxito/test.csv"
#@Archivos de entrenamiento y test

import numpy as np
import pydrive as drive
from google.colab import drive
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
drive.mount('/content/drive')
pd.set_option('display.max_rows', 3000)

data  = pd.read_csv(ruta_train, sep=",") #la columna 28 no tiene datos, marca error por eso, pero se eliminan los datos luego
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,...,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,...,HDD,299451.0,0,4096.0,Desktop,18.9,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,2668.0,91656.0,4.0,5.0,...,HDD,102385.0,0,4096.0,Notebook,13.9,1366.0,768.0,Mobile,NaN,1.000000e+00,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8.0,31,UNKNOWN,0,OFFLINE,Retail,NaN,0.0,NOT_SET,NaN,628.0,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,4909.0,317701.0,4.0,5.0,...,SSD,113907.0,0,4096.0,Desktop,21.5,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7.0,30,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,142.0,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,NaN,117.0,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,1443.0,275890.0,4.0,5.0,...,UNKNOWN,227116.0,0,4096.0,MiniTower,18.5,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,17.0,64,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,355.0,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417e

## **Revisión de valores perdidos**

In [2]:
def missing_values_table(df): 
        val_per = df.isnull().sum()
        porcentaje_val_per = 100 * df.isnull().sum()/len(df)
        tabla_val_per = pd.concat([val_per, porcentaje_val_per], axis=1)
        tabla_val_per_ren_columnas = tabla_val_per.rename(
        columns = {0 : 'Valores perdidos', 1 : '% sobre el total'})
        return tabla_val_per_ren_columnas
valores_perdidos = missing_values_table(data)
valores_perdidos = valores_perdidos.sort_values(by=['% sobre el total'], ascending=False)
valores_perdidos.head(valores_perdidos.size)

,Valores perdidos,% sobre el total
PuaMode,8919174,99.974119
Census_ProcessorClass,8884852,99.589407
DefaultBrowsersIdentifier,8488045,95.141637
Census_IsFlightingInternal,7408759,83.044030
Census_InternalBatteryType,6338429,71.046809
Census_ThresholdOptIn,5667325,63.524472
Census_IsWIMBootEnabled,5659703,63.439038
SmartScreen,3177011,35.610795
OrganizationIdentifier,2751518,30.841487
SMode,537759,6.027686


### Se eliminan las variables cuyos datos representen una pérdida de información en el conjunto de entrenamiento del 30% o superior.

In [3]:
for i in valores_perdidos.index:
  if valores_perdidos.loc[i].values[1] > 30:
    data = data.drop([i], axis=1)
    print('Se eliminó la variable:',i)
  else:
    break

Se eliminó la variable: PuaMode
Se eliminó la variable: Census_ProcessorClass
Se eliminó la variable: DefaultBrowsersIdentifier
Se eliminó la variable: Census_IsFlightingInternal
Se eliminó la variable: Census_InternalBatteryType
Se eliminó la variable: Census_ThresholdOptIn
Se eliminó la variable: Census_IsWIMBootEnabled
Se eliminó la variable: SmartScreen
Se eliminó la variable: OrganizationIdentifier


### Las perdidas de datos no son superiores al 7%, se eliminan todas las filas cuyos valores sean nulos en cualquier columna del conjunto de entrenamiento.

In [4]:
valores_perdidos = missing_values_table(data)
valores_perdidos = valores_perdidos.sort_values(by=['% sobre el total'], ascending=False)
data = data.dropna()
valores_perdidos.head(valores_perdidos.size)


,Valores perdidos,% sobre el total
SMode,537759,6.027686
CityIdentifier,325409,3.647477
Wdft_RegionIdentifier,303451,3.401352
Wdft_IsGamer,303451,3.401352
Census_InternalBatteryNumberOfCharges,268755,3.012448
Census_FirmwareManufacturerIdentifier,183257,2.054109
Census_IsFlightsDisabled,160523,1.799286
Census_FirmwareVersionIdentifier,160133,1.794915
Census_OEMModelIdentifier,102233,1.145919
Census_OEMNameIdentifier,95478,1.070203


In [5]:
valores_perdidos = missing_values_table(data)
valores_perdidos = valores_perdidos.sort_values(by=['% sobre el total'], ascending=False)
valores_perdidos.head(valores_perdidos.size)

,Valores perdidos,% sobre el total
MachineIdentifier,0,0.0
Census_OSInstallTypeName,0,0.0
Census_OSEdition,0,0.0
Census_OSBuildRevision,0,0.0
Census_OSBuildNumber,0,0.0
Census_OSBranch,0,0.0
Census_OSArchitecture,0,0.0
Census_OSVersion,0,0.0
Census_InternalBatteryNumberOfCharges,0,0.0
Census_PowerPlatformRoleName,0,0.0


## **Identificación de variables y valores únicos**

In [6]:
unicos_por_variable = []
for i in data.columns:
  unicos_por_variable.append([i, len(data[i].unique())])
unicos_por_variable = pd.DataFrame(data=unicos_por_variable, columns=['Variable', 'Valores únicos'])
unicos_por_variable = unicos_por_variable.sort_values(by=['Valores únicos'], ascending=True)
unicos_por_variable

,Variable,Valores únicos
73,HasDetections,2
26,SMode,2
28,Firewall,2
69,Census_IsPenCapable,2
62,Census_IsFlightsDisabled,2
11,HasTpm,2
68,Census_IsTouchEnabled,2
67,Census_IsVirtualDevice,2
70,Census_IsAlwaysOnAlwaysConnectedCapable,2
7,IsSxsPassiveMode,2


In [7]:
data.dtypes

MachineIdentifier                                     object
ProductName                                           object
EngineVersion                                         object
AppVersion                                            object
AvSigVersion                                          object
IsBeta                                                 int64
RtpStateBitfield                                     float64
IsSxsPassiveMode                                       int64
AVProductStatesIdentifier                            float64
AVProductsInstalled                                  float64
AVProductsEnabled                                    float64
HasTpm                                                 int64
CountryIdentifier                                      int64
CityIdentifier                                       float64
GeoNameIdentifier                                    float64
LocaleEnglishNameIdentifier                            int64
Platform                